In [0]:
-- =====================================================
-- DAY 10 POC - VIEWS & PERFORMANCE BASICS
-- My Learning Practice
-- Date: November 10, 2025
-- 
-- What I'm practicing today:
-- 1. Creating and using regular views
-- 2. Understanding materialized views
-- 3. Using EXPLAIN to check query performance
-- 4. Basic index concepts
-- 5. Query optimization techniques
-- =====================================================

-- Clean up from any previous practice
DROP VIEW IF EXISTS v_active_customers;
DROP VIEW IF EXISTS v_customer_summary;
DROP VIEW IF EXISTS v_high_value_orders;
DROP MATERIALIZED VIEW IF EXISTS mv_daily_sales;
DROP TABLE IF EXISTS practice_customers;
DROP TABLE IF EXISTS practice_orders;
DROP TABLE IF EXISTS practice_products;

-- =====================================================
-- SECTION 1: Setup Practice Tables
-- =====================================================

SELECT '=== Setting Up Practice Data ===' AS section;

-- Create customers table
CREATE TABLE practice_customers (
    customer_id INT PRIMARY KEY,
    name STRING,
    email STRING,
    city STRING,
    status STRING,
    signup_date DATE,
    total_purchases DECIMAL(10, 2)
);

-- Insert sample customers
INSERT INTO practice_customers VALUES
(101, 'Alice Johnson', 'alice@email.com', 'New York', 'Active', '2023-01-15', 5000.00),
(102, 'Bob Smith', 'bob@email.com', 'Los Angeles', 'Active', '2023-03-20', 12000.00),
(103, 'Carol White', 'carol@email.com', 'Chicago', 'Inactive', '2022-06-10', 3000.00),
(104, 'David Brown', 'david@email.com', 'New York', 'Active', '2024-01-05', 8000.00),
(105, 'Eve Davis', 'eve@email.com', 'Boston', 'Active', '2023-08-12', 15000.00),
(106, 'Frank Miller', 'frank@email.com', 'Chicago', 'Inactive', '2022-11-30', 2000.00);

-- Create orders table
CREATE TABLE practice_orders (
    order_id INT PRIMARY KEY,
    customer_id INT,
    order_date DATE,
    amount DECIMAL(10, 2),
    status STRING
);

-- Insert sample orders
INSERT INTO practice_orders VALUES
(1001, 101, '2024-01-10', 500.00, 'Completed'),
(1002, 101, '2024-02-15', 750.00, 'Completed'),
(1003, 102, '2024-01-20', 1200.00, 'Completed'),
(1004, 102, '2024-03-05', 800.00, 'Completed'),
(1005, 104, '2024-02-28', 600.00, 'Completed'),
(1006, 105, '2024-01-15', 2000.00, 'Completed'),
(1007, 105, '2024-03-10', 1500.00, 'Pending'),
(1008, 103, '2024-01-25', 300.00, 'Cancelled');

-- Create products table for JOIN practice
CREATE TABLE practice_products (
    product_id INT PRIMARY KEY,
    product_name STRING,
    category STRING,
    price DECIMAL(10, 2)
);

INSERT INTO practice_products VALUES
(201, 'Laptop', 'Electronics', 1200.00),
(202, 'Phone', 'Electronics', 800.00),
(203, 'Desk', 'Furniture', 350.00),
(204, 'Chair', 'Furniture', 200.00);

SELECT 'Practice data loaded successfully!' AS status;
SELECT 'Customers:' AS table_name, COUNT(*) AS row_count FROM practice_customers
UNION ALL
SELECT 'Orders:', COUNT(*) FROM practice_orders
UNION ALL
SELECT 'Products:', COUNT(*) FROM practice_products;

-- =====================================================
-- SECTION 2: CREATING SIMPLE VIEWS
-- Learning: Views are saved queries that act like tables
-- =====================================================

SELECT '=== DEMO 1: Simple View - Filter Rows ===' AS demo;

-- Create view showing only active customers
CREATE VIEW v_active_customers AS
SELECT 
    customer_id,
    name,
    email,
    city,
    total_purchases
FROM practice_customers
WHERE status = 'Active';

-- Now I can query the view like a table
SELECT 'Querying view - only active customers shown' AS note;
SELECT * FROM v_active_customers ORDER BY customer_id;

-- The view doesn't store data, it just runs this query:
-- SELECT * FROM practice_customers WHERE status = 'Active'

SELECT 'View Benefit 1' AS lesson,
       'Simplifies repeated queries - dont need to write WHERE clause every time' AS learned;

-- =====================================================
-- DEMO 2: Complex View - Joins and Aggregations
-- =====================================================

SELECT '=== DEMO 2: Complex View with JOIN and GROUP BY ===' AS demo;

-- Create view that summarizes customer orders
CREATE VIEW v_customer_summary AS
SELECT 
    c.customer_id,
    c.name,
    c.city,
    c.status,
    COUNT(o.order_id) AS order_count,
    COALESCE(SUM(o.amount), 0) AS total_order_amount,
    MAX(o.order_date) AS last_order_date
FROM practice_customers c
LEFT JOIN practice_orders o ON c.customer_id = o.customer_id
GROUP BY c.customer_id, c.name, c.city, c.status;

-- Query the view - much simpler than writing full JOIN every time!
SELECT 'Customer summary from view' AS note;
SELECT * FROM v_customer_summary ORDER BY total_order_amount DESC;

-- Without view, I would have to write the entire JOIN + GROUP BY every time
-- With view, I just SELECT FROM view_name

SELECT 'View Benefit 2' AS lesson,
       'Hides complexity - users query view without knowing underlying JOINs' AS learned;

-- =====================================================
-- DEMO 3: View for Security - Hide Sensitive Columns
-- =====================================================

SELECT '=== DEMO 3: View for Security ===' AS demo;

-- Create view that hides email addresses
CREATE VIEW v_customer_public AS
SELECT 
    customer_id,
    name,
    city,
    status
    -- Notice: email is NOT included (hidden for security)
FROM practice_customers;

SELECT 'Public view - email addresses hidden' AS note;
SELECT * FROM v_customer_public LIMIT 3;

-- Real scenario: Give reporting team access to v_customer_public
-- They can see names and cities, but NOT sensitive email addresses

SELECT 'View Benefit 3' AS lesson,
       'Security - can hide sensitive columns from certain users' AS learned;

-- =====================================================
-- DEMO 4: Filtering Views Further
-- =====================================================

SELECT '=== DEMO 4: Querying Views with Additional Filters ===' AS demo;

-- I can add WHERE clause when querying views
SELECT 'High-value active customers' AS note;
SELECT * FROM v_customer_summary
WHERE status = 'Active' AND total_order_amount > 1000
ORDER BY total_order_amount DESC;

-- View already did the JOIN and GROUP BY
-- I just added my specific filter on top

SELECT 'View Pattern' AS lesson,
       'Create view for complex base query, add specific filters when querying' AS learned;

-- =====================================================
-- SECTION 3: MATERIALIZED VIEWS
-- Learning: Unlike regular views, these STORE the data
-- =====================================================

SELECT '=== Understanding MATERIALIZED VIEWS ===' AS section;

-- Create materialized view for daily sales summary
-- Note: This actually STORES the aggregated data
CREATE MATERIALIZED VIEW mv_daily_sales AS
SELECT 
    order_date,
    COUNT(*) AS order_count,
    SUM(amount) AS total_sales,
    ROUND(AVG(amount), 2) AS avg_order_value
FROM practice_orders
WHERE status = 'Completed'
GROUP BY order_date;

SELECT 'Daily sales from materialized view (fast!)' AS note;
SELECT * FROM mv_daily_sales ORDER BY order_date;

-- Key difference: Regular view runs query every time
--                 Materialized view stores results (faster!)

SELECT 'Materialized View' AS concept,
       'Stores actual data - faster to query but needs refresh to see new data' AS explanation;

-- =====================================================
-- DEMO 5: Refreshing Materialized Views
-- =====================================================

SELECT '=== DEMO 5: Data Staleness in Materialized Views ===' AS demo;

-- Add new order
INSERT INTO practice_orders VALUES
(1009, 104, CURRENT_DATE(), 900.00, 'Completed');

SELECT 'Added new order for today' AS note;
SELECT * FROM practice_orders WHERE order_id = 1009;

-- Query materialized view - NEW order is NOT shown yet!
SELECT 'Querying mv_daily_sales (new order NOT visible yet)' AS note;
SELECT * FROM mv_daily_sales WHERE order_date = CURRENT_DATE();
-- Returns nothing because materialized view wasn't refreshed

-- To see new data, must REFRESH materialized view
REFRESH MATERIALIZED VIEW mv_daily_sales;

SELECT 'After refresh (now new order is visible)' AS note;
SELECT * FROM mv_daily_sales WHERE order_date = CURRENT_DATE();

-- Now today's order appears!

SELECT 'Materialized View Lesson' AS lesson,
       'Data can be stale. Must refresh to see new data. Trade-off: speed vs freshness' AS learned;

-- =====================================================
-- SECTION 4: EXPLAIN - Understanding Query Performance
-- Learning: EXPLAIN shows HOW database executes query
-- =====================================================

SELECT '=== Understanding EXPLAIN ===' AS section;

-- EXPLAIN shows the execution plan
SELECT 'Using EXPLAIN to see query execution plan' AS note;

EXPLAIN
SELECT * FROM practice_customers WHERE city = 'New York';

-- Output shows:
-- - Scan type (Sequential Scan or Index Scan)
-- - Filter conditions
-- - Estimated cost
-- - Estimated rows

SELECT 'EXPLAIN Lesson' AS lesson,
       'Shows how database will execute query. Use to find performance issues.' AS learned;

-- =====================================================
-- DEMO 6: Sequential Scan vs Index Scan
-- =====================================================

SELECT '=== DEMO 6: Seeing Sequential Scan ===' AS demo;

-- This query does Sequential Scan (reads entire table)
EXPLAIN
SELECT * FROM practice_customers WHERE city = 'Chicago';

-- Sequential Scan means: Read every row, check if city = 'Chicago'
-- For small tables: okay
-- For millions of rows: SLOW!

SELECT 'Sequential Scan' AS scan_type,
       'Reads entire table row by row. Slow for large tables.' AS explanation;

-- =====================================================
-- DEMO 7: Creating Index for Better Performance
-- =====================================================

SELECT '=== DEMO 7: Creating Index to Speed Up Queries ===' AS demo;

-- Create index on city column
CREATE INDEX idx_city ON practice_customers(city);

SELECT 'Created index on city column' AS note;

-- Now EXPLAIN again - should use Index Scan
EXPLAIN
SELECT * FROM practice_customers WHERE city = 'Chicago';

-- After index, should see "Index Scan" instead of "Sequential Scan"
-- Index Scan: Jump directly to matching rows (much faster!)

SELECT 'Index Benefit' AS lesson,
       'Queries using indexed columns are much faster. Like using book index vs reading entire book.' AS learned;

-- =====================================================
-- DEMO 8: Index on JOIN Columns
-- =====================================================

SELECT '=== DEMO 8: Indexes on JOIN Columns ===' AS demo;

-- Check execution plan for JOIN query
EXPLAIN
SELECT c.name, o.order_date, o.amount
FROM practice_customers c
JOIN practice_orders o ON c.customer_id = o.customer_id;

-- Create index on foreign key for better JOIN performance
CREATE INDEX idx_orders_customer_id ON practice_orders(customer_id);

SELECT 'Created index on orders.customer_id (foreign key)' AS note;

-- Now EXPLAIN again - should be faster
EXPLAIN
SELECT c.name, o.order_date, o.amount
FROM practice_customers c
JOIN practice_orders o ON c.customer_id = o.customer_id;

SELECT 'JOIN Performance' AS lesson,
       'Always index foreign key columns used in JOINs' AS learned;

-- =====================================================
-- SECTION 5: Query Optimization Techniques I Learned
-- =====================================================

SELECT '=== OPTIMIZATION TECHNIQUE 1: Select Only Needed Columns ===' AS technique;

-- BAD: Select all columns
SELECT 'Bad practice - SELECT *' AS example;
EXPLAIN
SELECT * FROM practice_customers;

-- GOOD: Select only what you need
SELECT 'Good practice - Select specific columns' AS example;
EXPLAIN
SELECT customer_id, name, city FROM practice_customers;

-- Selecting fewer columns = less data transfer = faster

SELECT 'Optimization 1' AS technique,
       'SELECT only columns you need. Avoid SELECT * in production.' AS learned;

-- =====================================================
-- TECHNIQUE 2: Use LIMIT for Large Results
-- =====================================================

SELECT '=== OPTIMIZATION TECHNIQUE 2: Use LIMIT ===' AS technique;

-- Without LIMIT - returns all rows (could be millions!)
SELECT 'Without LIMIT - could return millions of rows' AS example;

-- With LIMIT - only returns what you need
SELECT 'With LIMIT - only get top N rows' AS example;
SELECT * FROM practice_orders 
ORDER BY order_date DESC 
LIMIT 5;

SELECT 'Optimization 2' AS technique,
       'Use LIMIT when you dont need all rows. Especially for large tables.' AS learned;

-- =====================================================
-- TECHNIQUE 3: Filter Early
-- =====================================================

SELECT '=== OPTIMIZATION TECHNIQUE 3: Filter Early ===' AS technique;

-- LESS EFFICIENT: JOIN all then filter
SELECT 'Less efficient - filter after JOIN' AS example;
SELECT c.name, o.amount
FROM practice_customers c
JOIN practice_orders o ON c.customer_id = o.customer_id
WHERE c.city = 'New York';

-- MORE EFFICIENT: Filter before JOIN (fewer rows to join)
SELECT 'More efficient - filter before JOIN' AS example;
SELECT c.name, o.amount
FROM (SELECT * FROM practice_customers WHERE city = 'New York') c
JOIN practice_orders o ON c.customer_id = o.customer_id;

SELECT 'Optimization 3' AS technique,
       'Filter data early to reduce rows in JOINs. Fewer rows = faster.' AS learned;

-- =====================================================
-- TECHNIQUE 4: Avoid Functions on Indexed Columns
-- =====================================================

SELECT '=== OPTIMIZATION TECHNIQUE 4: Avoid Functions on Indexed Columns ===' AS technique;

-- BAD: Function on indexed column (cant use index)
SELECT 'Bad - function prevents index usage' AS example;
EXPLAIN
SELECT * FROM practice_customers 
WHERE UPPER(city) = 'NEW YORK';

-- GOOD: No function (can use index)
SELECT 'Good - can use index' AS example;
EXPLAIN
SELECT * FROM practice_customers 
WHERE city = 'New York';

SELECT 'Optimization 4' AS technique,
       'Avoid functions on indexed columns. Prevents index usage.' AS learned;

-- =====================================================
-- SECTION 6: View Management Commands
-- =====================================================

SELECT '=== View Management Commands I Learned ===' AS section;

-- Show view definition
SELECT 'To see view